# YOLOv9 License Plate Detection Pipeline

This notebook provides a complete, end-to-end pipeline for training a custom YOLOv9 (GELAN-C) object detection model to detect license plates. It handles the entire process from environment setup to model export.

### Key Steps in this Notebook:

1. Environment Setup:

    * Clones the official YOLOv9 repository.

    * Installs necessary dependencies.

    * Critical Patches: Automatically applies patches to source files to fix compatibility issues with PyTorch 2.6+ (specifically the weights_only=False security update).

2. Dataset Preparation:

    * Downloads a labeled License Plate dataset from Roboflow.

    * Automatically rewrites the data.yaml configuration file to use absolute paths, preventing common "Dataset not found" errors in Colab.

3. Training:

    * Fine-tunes the pre-trained gelan-c model on the custom dataset.

    * Disables W&B logging to ensure uninterrupted training.

4. Validation & Inference:

    * Performs manual cleanup of the model weights to strip unnecessary optimizer data.

    * Runs batch detection on test images to visualize performance immediately.

5. Export:

    * Converts the final PyTorch (.pt) model into ONNX format, ready for deployment in high-performance web applications (like FastAPI).

    * Zips and downloads the trained weights to your local machine.

# Environmental Setup

In [ ]:
# Check GPU Status
!nvidia-smi

In [ ]:
# Clone YOLOv9 Repository
!git clone https://github.com/WongKinYiu/yolov9.git

In [ ]:
# Install Dependencies (including Pillow fix and ONNX support)
!pip install -r requirements.txt
!pip install roboflow
!pip install pillow==9.5.0
!pip install onnxscript onnx

In [ ]:
# Download Pre-trained Weights (GELAN-C)
!wget -P /content/yolov9 https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P /content/yolov9 https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-c.pt

In [ ]:
# Apply PyTorch 2.6+ Security Patches (Fixes 'weights_only' errors globally)
# Patch train.py
!sed -i "s/torch.load(weights, map_location='cpu')/torch.load(weights, map_location='cpu', weights_only=False)/" /content/yolov9/train.py
# Patch detect.py
!sed -i "s/torch.load(weights, map_location=device)/torch.load(weights, map_location=device, weights_only=False)/" /content/yolov9/detect.py
# Patch export.py and experimental.py
!sed -i "s/torch.load(attempt_download(w), map_location='cpu')/torch.load(attempt_download(w), map_location='cpu', weights_only=False)/" /content/yolov9/models/experimental.py
# Patch utils/general.py
!sed -i "s/torch.load(f, map_location=torch.device('cpu'))/torch.load(f, map_location=torch.device('cpu'), weights_only=False)/" /content/yolov9/utils/general.py

print("✅ Environment Setup Complete & Scripts Patched!")

# Dataset Preparation

In [ ]:
from roboflow import Roboflow
import yaml
import os

In [ ]:
# Download Dataset
rf = Roboflow(api_key="")
project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
version = project.version(11)
dataset = version.download("yolov9")

In [ ]:
# Fix paths in data.yaml
dataset_dir = '/content/yolov9/License-Plate-Recognition-11'
yaml_path = os.path.join(dataset_dir, 'data.yaml')

In [ ]:
# Load the current yaml
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

print(f"Old paths: Train={data.get('train')}, Val={data.get('val')}")

In [ ]:
# Overwrite with absolute paths
data['train'] = os.path.join(dataset_dir, 'train/images')
data['val'] = os.path.join(dataset_dir, 'valid/images')
data['test'] = os.path.join(dataset_dir, 'test/images')

In [ ]:
# Save the file back
with open(yaml_path, 'w') as f:
    yaml.dump(data, f)

print(f"New paths: Train={data['train']}, Val={data['val']}")
print("✅ data.yaml paths fixed!")

# Model Training

In [ ]:
# Disable W&B logging
!wandb disabled

In [ ]:
# Run Training
!python train.py \
--workers 8 \
--device 0 \
--batch 16 \
--data {dataset.location}/data.yaml \
--img 640 \
--cfg models/detect/gelan-c.yaml \
--weights /content/yolov9/gelan-c.pt \
--name gelan-c-license-plate \
--hyp hyp.scratch-high.yaml \
--min-items 0 \
--epochs 1 \
--close-mosaic 10

# Model Post-Processing (Cleanup)

In [ ]:
import torch
import os
import glob

In [ ]:
# Find latest trained model
list_of_files = glob.glob('/content/yolov9/runs/train/**/best.pt', recursive=True)
latest_weights = max(list_of_files, key=os.path.getctime)
print(f"Found model at: {latest_weights}")

In [ ]:
# Manually Strip the Optimizer
try:
    # Load with the security fix
    ckpt = torch.load(latest_weights, map_location='cpu', weights_only=False)

    # Remove massive optimizer data (makes file 75% smaller)
    ckpt['optimizer'] = None
    ckpt['training_results'] = None
    ckpt['updates'] = None

    # Save it back
    torch.save(ckpt, latest_weights)
    print("✅ Model successfully cleaned and saved! You are ready for detection.")

except Exception as e:
    print(f"⚠️ Manual strip failed (files might already be clean), but you can likely still use the model. Error: {e}")

# Batch Inference (Testing)

In [ ]:
import os
import glob
import shutil
from IPython.display import Image, display

In [ ]:
# Prepare the images
# Create a temporary folder and copy listed images into it
source_images = [
    "/content/Vehicle_with__license_plate_01.jpg",
    "/content/Vehicle_with__license_plate_02.jpg",
    "/content/Vehicle_with__license_plate_03.jpg",
    "/content/Vehicle_with__license_plate_04.jpg",
    "/content/Vehicle_with__license_plate_05.jpg"
]

temp_folder = "/content/inference_batch"
os.makedirs(temp_folder, exist_ok=True)

# Copy images to the folder
for img_path in source_images:
    if os.path.exists(img_path):
        shutil.copy(img_path, temp_folder)
    else:
        print(f"⚠️ Warning: Could not find {img_path}, skipping...")

In [ ]:
# Run Detection on the FOLDER
print(f"🚀 Running detection on folder: {temp_folder}...")
!python detect.py \
--img 640 \
--conf 0.4 \
--device 0 \
--weights {latest_weights} \
--source {temp_folder}

In [ ]:
# Display ALL Results
# Find the latest experiment folder
latest_det = max([os.path.join('runs/detect', d) for d in os.listdir('runs/detect')], key=os.path.getmtime)

# Get all image files (jpg, jpeg, webp, png)
result_files = []
for ext in ('*.jpg', '*.jpeg', '*.webp', '*.png'):
    result_files.extend(glob.glob(os.path.join(latest_det, ext)))

if result_files:
    print(f"✅ Detection successful! Found {len(result_files)} images.")
    for img_file in result_files:
        print(f"Displaying: {os.path.basename(img_file)}")
        display(Image(filename=img_file))
else:
    print("❌ Detection ran, but no output images were found.")

# Export to ONNX

In [ ]:
print(f"🚀 Starting Export for {latest_weights}...")

!python export.py \
--weights {latest_weights} \
--include onnx \
--device cpu

# Download Model Files

In [ ]:
import shutil
from google.colab import files

In [ ]:
# Define paths
folder_path = os.path.dirname(latest_weights)
output_zip = "/content/my_license_plate_model.zip"

In [ ]:
# Zip the weights folder
shutil.make_archive("/content/my_license_plate_model", 'zip', folder_path)

In [ ]:
# Trigger download
print("Downloading model files...")
files.download(output_zip)